# Homology of graphs



#### Challenge 1

A multigraph can be transformed into a directed multigraph by arbitrarily assigning a direction to its edges.  
Prove that the resulting homology (is up to isomorphism) independent of the assigned directions.

---

**Answer:**  

Consider an undirected multigraph $G = (V, E)$. By arbitrarily assigning an orientation to each edge, we obtain a directed multigraph. Let $\partial$ be the boundary map defined on a basis element $e$ (oriented from $v$ to $w$) by
$$
\partial(e) = w - v.
$$

Suppose we have two choices of orientations that yield two boundary maps, $\partial$ and $\partial'$. For any edge $e$:
- If the orientation is the same in both choices, then $\partial'(e) = \partial(e)$.
- If the orientation is reversed, then $\partial'(e) = -\partial(e)$.

Define the isomorphism $\phi: C_1 \to C_1'$ by
$$
\phi(e) = \epsilon(e) \, e,
$$
where
$$
\epsilon(e) =
\begin{cases}
1, & \text{if the orientations agree,} \\
-1, & \text{if the orientations are reversed.}
\end{cases}
$$

Then, for any edge $e$,
$$
\partial'(\phi(e)) = \partial'(\epsilon(e)e) = \epsilon(e) \, \partial'(e).
$$
Since $\partial'(e) = \pm \partial(e)$ according to the orientation, we have
$$
\epsilon(e) \, \partial'(e) = \partial(e).
$$
Thus, the following diagram commutes:
$$
\begin{array}{ccc}
C_1 & \xrightarrow{\phi} & C_1' \\
\partial \downarrow & & \downarrow \partial' \\
C_0 & = & C_0
\end{array}
$$

Since the chain complexes are isomorphic via $\phi$, the homology groups
$$
H_0 = C_0 / \operatorname{img}(\partial) \quad \text{and} \quad H_1 = \ker(\partial)
$$
are isomorphic regardless of the chosen orientations.

Therefore, the homology (up to isomorphism) is independent of the arbitrary choices of edge orientations.


## Interpretation

Intuitively:
- $H_0$ encodes the connected components of the graph.
- $H_1$ encodes the independent loops in the graph.

The following challenges make these statements precise.

---

#### Challenge 2

Prove that two vertices belong to the same homology class if and only if they are part of the same connected component.

This implies that the dimension of $H_0$ corresponds to the number of connected components in the graph.

---

**Answer**

Let $G = (V, E)$ be a directed multigraph, where $C_0$ is the vector space spanned by the vertices and the boundary map $\partial \colon C_1 \to C_0$ is defined by
$$
\partial(v,w) = w - v.
$$
The degree 0 homology is
$$
H_0 = C_0 / \operatorname{img}(\partial).
$$

**(⇒)** Suppose two vertices $v$ and $w$ are in the same connected component. Then there is a path
$$
v = v_0, v_1, \dots, v_n = w
$$
with each $(v_i, v_{i+1}) \in E$. Since $\partial(v_i,v_{i+1}) = v_{i+1} - v_i$, we have
$$
v_{i+1} \equiv v_i \quad (\text{mod } \operatorname{img}(\partial)).
$$
Chaining these relations yields $v \equiv w$ in $H_0$, meaning they belong to the same homology class.

**(⇐)** Conversely, if $v$ and $w$ are in the same homology class, then
$$
v - w \in \operatorname{img}(\partial).
$$
Thus, there exists a finite linear combination of boundaries
$$
v - w = \sum_i a_i (x_i - y_i),
$$
with each $(x_i, y_i)$ an edge in $G$. This implies that $v$ and $w$ are connected by a chain of edges, so they lie in the same connected component.

Therefore, two vertices are homologous if and only if they belong to the same connected component, which shows that the dimension of $H_0$ equals the number of connected components in $G$.



---
#### Challenge 3

Prove that any undirected loop defines a homology class and that every class in $H_1$ can be expressed as a linear combination of these classes.

This implies that the dimension of $H_1$ equals the number of linearly independent loops in the graph.

---

**Claim 1:** Any undirected loop defines a homology class in $H_1$.  
**Proof:**  
Let $v_0, v_1, \dots, v_{n-1}, v_n = v_0$ be a loop in $G$, and consider the chain
$$
c = (v_0, v_1) + (v_1, v_2) + \cdots + (v_{n-1}, v_n).
$$
Applying the boundary map,
$$
\partial(c) = (v_1 - v_0) + (v_2 - v_1) + \cdots + (v_n - v_{n-1}) = v_n - v_0 = 0.
$$
Thus, $c \in \ker(\partial)$ and represents a homology class in $H_1$.

---

**Claim 2:** Every class in $H_1$ can be expressed as a linear combination of homology classes defined by undirected loops.  
**Proof:**  
Let $z \in \ker(\partial)$ be an arbitrary cycle in $C_1$. Write
$$
z = \sum_{e \in E} a_e e, \quad a_e \in \mathbb{F}.
$$
The condition $\partial(z)=0$ forces the net contribution at each vertex to vanish, capturing the idea of a "circulation" around loops. If $z$ contains repeated vertices, one can identify a subchain that forms a loop. Removing this subchain (and adjusting the coefficients) produces another cycle. Repeating this process, $z$ decomposes as a sum of cycles each supported on a simple loop.  
Hence, every cycle in $H_1$ is a linear combination of cycles coming from undirected loops.

---


## Boundary matrix

Let $B$ be the matrix representating $\partial$ with respect to the canonical bases $V$ and $E$ ordered arbitrarily.

Since $\operatorname{dim}\mathrm{img}(\partial) = \operatorname{rank}(B)$ we have

$$
\operatorname{dim} H_0 = |V| - \operatorname{rank}(B).
$$

The rank-nullity theorem gives

$$
\operatorname{dim} H_1 = |E| - \operatorname{rank}(B).
$$

---

#### Challenge 2

Observe that the Euler characteristic of a multigraph $G$ is equal to

$$
\chi(G) = \operatorname{dim} H_0 - \operatorname{dim} H_1.
$$

---

**Answer**

We have the following identities:
$$
\operatorname{dim} H_0 = |V| - \operatorname{rank}(B)
$$
and, by the rank-nullity theorem,
$$
\operatorname{dim} H_1 = |E| - \operatorname{rank}(B).
$$

Subtracting these, we obtain:
$$
\operatorname{dim} H_0 - \operatorname{dim} H_1 = \big(|V| - \operatorname{rank}(B)\big) - \big(|E| - \operatorname{rank}(B)\big) = |V| - |E|.
$$

Thus, the Euler characteristic is
$$
\chi(G) = \operatorname{dim} H_0 - \operatorname{dim} H_1 = |V| - |E|.
$$

--- 

## Functoriality

An important property of the homology construction is that it is *functorial*.
For our purposes, the most important special case of this property is the following:

**Theorem** If $G_{sub}$ is a directed submultigraph of $G$, then, for any degree $k \in \{0, 1\}$, the inclusion of vertices and edges induces a linear map

$$
\iota_k \colon H_k(G_{sub}) \to H_k(G).
$$

*Proof*. Since all the vertices and edges of $G_{sub}$ are vertices and edges of $G$ there is a natural inclusion of $C_k(G_{sub})$ into $C_k(G)$.
Any cycle in $C_1(G_{sub})$ is a cycle in $C_1(G)$, which defines the above inclusion for $k = 1$.
For $k = 0$, we must show that the map induced on chains preserves equivalence classes.
Since all edges of $H$ are in $G$, we have that $\operatorname{img} \partial_{sub} \subseteq \operatorname{img} \partial$.
So, given two representatives $c$ and $c'$ of the same homology class of $G_{sub}$, explicitly $c-c' \in \operatorname{img} \partial_{sub}$, they represent the same homology class in $G$ since $c-c' \in \operatorname{img} \partial$. &#9633;

---

#### Challenge 4

Show that for any three directed multigraphs $G \subseteq G' \subseteq G''$ and $k \in \{0, 1\}$ the map $H_k(G'') \to H_k(G)$ is equal to the composition of $H_k(G'') \to H_k(G')$ and $H_k(G') \to H_k(G)$.

---

**Anwer**

Let $G \subseteq G' \subseteq G''$ be three directed multigraphs. The inclusion of vertices and edges induces natural chain maps 
$$
\iota_k \colon C_k(G'') \to C_k(G')
$$
and
$$
\iota_k \colon C_k(G') \to C_k(G)
$$
for $k \in \{0,1\}$. Since these inclusions are just restrictions of the natural inclusion 
$$
\iota_k \colon C_k(G'') \to C_k(G),
$$
we have that the composition
$$
\iota_k \colon C_k(G'') \to C_k(G') \to C_k(G)
$$
is equal to the inclusion map 
$$
\iota_k \colon C_k(G'') \to C_k(G).
$$

Because the boundary maps are compatible with these inclusions (i.e. they make the diagram commute), the induced maps on homology also compose accordingly:
$$
H_k(G'') \to H_k(G') \to H_k(G)
$$
is equal to the map
$$
H_k(G'') \to H_k(G).
$$

Thus, for any three directed multigraphs $G \subseteq G' \subseteq G''$ and for $k \in \{0,1\}$, the map 
$$
H_k(G'') \to H_k(G)
$$
is the composition of 
$$
H_k(G'') \to H_k(G')
$$
and
$$
H_k(G') \to H_k(G).
$$

---

#### Challenge 4

Show that edge contraction preserves homology.
More explicitly, if $G'$ is obtained by contracting an edge of $G$, construct the natural function from the vertices of $G$ to the vertices of $G'$ and from the edges of $G$ to the edges of $G'$.
Notice that the induced map in homology is well-defined and a linear isomorphism.

---
**Answer**

Let $G = (V, E)$ be a directed multigraph and let $e_0 = (v, w)$ be an edge to be contracted. Denote by $G'$ the graph obtained by contracting $e_0$. Contraction identifies the vertices $v$ and $w$ to a single vertex, say $u'$. Define the natural maps as follows:

- **On vertices:** Define 
  $$
  f \colon V \to V' \quad \text{by} \quad f(x) =
  \begin{cases}
  u' & \text{if } x = v \text{ or } x = w,\\[1mm]
  x & \text{otherwise.}
  \end{cases}
  $$
- **On edges:** Define 
  $$
  f \colon E \to E' \quad \text{by} \quad f((x,y)) = (f(x), f(y)).
  $$
  Notice that $f(e_0) = (f(v), f(w)) = (u', u')$ is a degenerate edge and is omitted from $E'$.

These maps extend linearly to chain maps:
$$
f_0 \colon C_0(G) \to C_0(G') \quad \text{and} \quad f_1 \colon C_1(G) \to C_1(G'),
$$
and they satisfy
$$
f_0(\partial(e)) = \partial(f_1(e))
$$
for every edge $e \in E$. Hence, they form a chain map between the chain complexes of $G$ and $G'$.

The induced homomorphism in homology,
$$
f_\ast \colon H_k(G) \to H_k(G'),
$$
is well-defined for $k \in \{0,1\}$. Moreover, since edge contraction does not change the connectivity of the graph or the presence of cycles (aside from eliminating the degenerate cycle corresponding to $e_0$), $f_\ast$ is a linear isomorphism. In other words, contracting an edge preserves both $H_0$ and $H_1$.

---

#### Challenge 5

Show that $\beta_1(G)$, the first Betti number of $G$, which was defined in the previous notebook as the number of edges of the minimal model of $G$, agrees with the dimension of $H_1(G)$.

---

**Answer**

Recall that the first Betti number $\beta_1(G)$ was defined as the number of edges in the minimal model of $G$, which corresponds to the number of independent cycles. More concretely, if $G$ has $|V|$ vertices and $|E|$ edges, and if $G$ has $k$ connected components, then a spanning forest of $G$ contains $|V| - k$ edges. The number of extra edges, which generate independent cycles, is
$$
\beta_1(G) = |E| - (|V| - k).
$$

On the other hand, from the homology point of view we have
$$
\dim H_0(G) = |V| - \operatorname{rank}(B),
$$
and by the rank-nullity theorem,
$$
\dim H_1(G) = |E| - \operatorname{rank}(B).
$$
Since the boundary map $B$ sends an edge $(x,y)$ to $y - x$, the image of $B$ is generated by differences of vertices, and we have
$$
\operatorname{rank}(B) = |V| - k.
$$
Thus,
$$
\dim H_1(G) = |E| - (|V| - k) = \beta_1(G).
$$

This shows that the first Betti number $\beta_1(G)$ agrees with the dimension of $H_1(G)$.